# <center> Learned primal-dual</center>

Import libraries.

In [1]:
import os
import adler
adler.util.gpu.setup_one_gpu(0)

Picking GPU 0


In [2]:
from adler.odl.phantom import random_phantom
from adler.tensorflow import prelu, cosine_decay

In [3]:
import tensorflow as tf
import numpy as np
import odl
import odl.contrib.tensorflow

In [4]:
np.random.seed(0)
#name = os.path.splitext(os.path.basename(__file__))[0]
name = os.path.splitext(os.getcwd())[0]+'/checkpoints'

In [5]:
# Creating checkpoints directory
try:
    os.mkdir(name)
except OSError:
    print ("Creation of the directory %s failed" % name)
else:
    print ("Successfully created the directory %s " % name)

Creation of the directory /store/kepler/datastore/andrade/GitHub_repos/Joint_CTWF_Recon/Microcanonical_NN/checkpoints failed


In [6]:
sess = tf.InteractiveSession()

Define space and geometry

In [7]:
# Create ODL data structures
size = 128
space = odl.uniform_discr([-64, -64], [64, 64], [size, size],
                          dtype='float32')

In [8]:
geometry = odl.tomo.parallel_beam_geometry(space, num_angles=30)

In [9]:
operator = odl.tomo.RayTransform(space, geometry)

In [10]:
# Ensure operator has fixed operator norm for scale invariance
opnorm = odl.power_method_opnorm(operator)
operator = (1 / opnorm) * operator

In [11]:
operator.operator

RayTransform: uniform_discr([-64., -64.], [ 64.,  64.], (128, 128), dtype='float32') -> uniform_discr([  0.    , -90.5097], [  3.1416,  90.5097], (30, 183), dtype='float32', weighting=1.0)

In [12]:
# Create tensorflow layer from odl operator
odl_op_layer = odl.contrib.tensorflow.as_tensorflow_layer(operator,
                                                          'RayTransform')
odl_op_layer_adjoint = odl.contrib.tensorflow.as_tensorflow_layer(operator.adjoint,
                                                                  'RayTransformAdjoint')

In [13]:
# User selected paramters
n_data = 5
n_iter = 10
n_primal = 5
n_dual = 5

In [14]:
def generate_data(validation=False):
    """Generate a set of random data."""
    n_generate = 1 if validation else n_data

    y_arr = np.empty((n_generate, operator.range.shape[0], operator.range.shape[1], 1), dtype='float32')
    x_true_arr = np.empty((n_generate, space.shape[0], space.shape[1], 1), dtype='float32')

    for i in range(n_generate):
        if validation:
            phantom = odl.phantom.shepp_logan(space, True)
        else:
            phantom = random_phantom(space)
        data = operator(phantom)
        noisy_data = data + odl.phantom.white_noise(operator.range) * np.mean(np.abs(data)) * 0.05

        x_true_arr[i, ..., 0] = phantom
        y_arr[i, ..., 0] = noisy_data

    return y_arr, x_true_arr

In [15]:
with tf.name_scope('placeholders'):
    x_true = tf.placeholder(tf.float32, shape=[None, size, size, 1], name="x_true")
    y_rt = tf.placeholder(tf.float32, shape=[None, operator.range.shape[0], operator.range.shape[1], 1], name="y_rt")
    is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

In [16]:
x_true

<tf.Tensor 'placeholders/x_true:0' shape=(?, 128, 128, 1) dtype=float32>

In [17]:
y_arr, x_true_arr = generate_data()

In [18]:
x_true

<tf.Tensor 'placeholders/x_true:0' shape=(?, 128, 128, 1) dtype=float32>

In [19]:
x_true_arr.shape

(5, 128, 128, 1)

In [20]:
def apply_conv(x, filters=32):
    return tf.layers.conv2d(x, filters=filters, kernel_size=3, padding='SAME',
                            kernel_initializer=tf.contrib.layers.xavier_initializer())

In [21]:
with tf.name_scope('tomography'):
    with tf.name_scope('initial_values'):
        primal = tf.concat([tf.zeros_like(x_true)] * n_primal, axis=-1)
        dual = tf.concat([tf.zeros_like(y_rt)] * n_dual, axis=-1)

    for i in range(n_iter):
        with tf.variable_scope('dual_iterate_{}'.format(i)):
            evalop = odl_op_layer(primal[..., 1:2])
            update = tf.concat([dual, evalop, y_rt], axis=-1)

            #update = prelu(apply_conv(update), name='prelu_1')
            #update = prelu(apply_conv(update), name='prelu_2')
            update = tf.sigmoid(apply_conv(update))
            update = tf.sigmoid(apply_conv(update))
            update = apply_conv(update, filters=n_dual)
            dual = dual + update

        with tf.variable_scope('primal_iterate_{}'.format(i)):
            evalop = odl_op_layer_adjoint(dual[..., 0:1])
            update = tf.concat([primal, evalop], axis=-1)

            #update = prelu(apply_conv(update), name='prelu_1')
            #update = prelu(apply_conv(update), name='prelu_2')
            update = tf.sigmoid(apply_conv(update))
            update = tf.sigmoid(apply_conv(update))
            update = apply_conv(update, filters=n_primal)
            primal = primal + update

    x_result = primal[..., 0:1]

In [22]:
x_result

<tf.Tensor 'tomography/strided_slice:0' shape=(?, 128, 128, 1) dtype=float32>

In [23]:
name

'/store/kepler/datastore/andrade/GitHub_repos/Joint_CTWF_Recon/Microcanonical_NN/checkpoints'

In [24]:
with tf.name_scope('loss'):
    residual = x_result - x_true
    squared_error = residual ** 2
    loss = tf.reduce_mean(squared_error)

In [25]:
with tf.name_scope('optimizer'):
    # Learning rate
    global_step = tf.Variable(0, trainable=False)
    maximum_steps = 100001
    starter_learning_rate = 1e-3
    learning_rate = cosine_decay(starter_learning_rate,
                                 global_step,
                                 maximum_steps,
                                 name='learning_rate')

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt_func = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                          beta2=0.99)

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 1)
        optimizer = opt_func.apply_gradients(zip(grads, tvars),
                                             global_step=global_step)

In [26]:
ckp_name = name+'_lpd_sigmoid/checkpoints'

In [27]:
# Summaries
# tensorboard --logdir=...

with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('psnr', -10 * tf.log(loss) / tf.log(10.0))

    tf.summary.image('x_result', x_result)
    tf.summary.image('x_true', x_true)
    tf.summary.image('squared_error', squared_error)
    tf.summary.image('residual', residual)

    merged_summary = tf.summary.merge_all()
    test_summary_writer = tf.summary.FileWriter(adler.tensorflow.util.default_tensorboard_dir(ckp_name) + '/test',
                                                sess.graph)
    train_summary_writer = tf.summary.FileWriter(adler.tensorflow.util.default_tensorboard_dir(ckp_name) + '/train')

In [28]:
# Initialize all TF variables
sess.run(tf.global_variables_initializer())

In [29]:
# Add op to save and restore
saver = tf.train.Saver()

In [30]:
# Generate validation data
y_arr_validate, x_true_arr_validate = generate_data(validation=True)

In [31]:
if 1:
    saver.restore(sess,
                  adler.tensorflow.util.default_checkpoint_path(ckp_name))

INFO:tensorflow:Restoring parameters from /store/kepler/datastore/andrade/GitHub_repos/Joint_CTWF_Recon/Microcanonical_NN/checkpoints_lpd_sigmoid/checkpoints.ckpt


Train the network

In [32]:
# Train the network
for i in range(0, maximum_steps):
    if i%10 == 0:
        y_arr, x_true_arr = generate_data()

    _, merged_summary_result_train, global_step_result = sess.run([optimizer, merged_summary, global_step],
                              feed_dict={x_true: x_true_arr,
                                         y_rt: y_arr,
                                         is_training: True})

    if i>0 and i%10 == 0:
        loss_result, merged_summary_result, global_step_result = sess.run([loss, merged_summary, global_step],
                              feed_dict={x_true: x_true_arr_validate,
                                         y_rt: y_arr_validate,
                                         is_training: False})

        train_summary_writer.add_summary(merged_summary_result_train, global_step_result)
        test_summary_writer.add_summary(merged_summary_result, global_step_result)

        print('iter={}, loss={}'.format(global_step_result, loss_result))

    if i>0 and i%1000 == 0:
        saver.save(sess,
                   adler.tensorflow.util.default_checkpoint_path(ckp_name))

iter=114013, loss=0.00045187160139903426
iter=114023, loss=0.0004384742642287165
iter=114033, loss=0.000509448756929487
iter=114043, loss=0.0005139550776220858
iter=114053, loss=0.0005847396096214652
iter=114063, loss=0.0005302742938511074
iter=114073, loss=0.00043185794493183494
iter=114083, loss=0.0005591966910287738
iter=114093, loss=0.00040150724817067385
iter=114103, loss=0.0003281898098066449
iter=114113, loss=0.00033129286020994186
iter=114123, loss=0.0003254400799050927
iter=114133, loss=0.0003871559747494757
iter=114143, loss=0.0003194097662344575
iter=114153, loss=0.00025394908152520657
iter=114163, loss=0.00024924849276430905
iter=114173, loss=0.0002583123277872801
iter=114183, loss=0.0002875150239560753
iter=114193, loss=0.0002611529780551791
iter=114203, loss=0.00026643741875886917
iter=114213, loss=0.00031282968120649457
iter=114223, loss=0.0002926476299762726
iter=114233, loss=0.00027720368234440684
iter=114243, loss=0.0002715669688768685
iter=114253, loss=0.000272172153

iter=116043, loss=0.0003197863115929067
iter=116053, loss=0.0003012418164871633
iter=116063, loss=0.0003279096563346684
iter=116073, loss=0.000317811849527061
iter=116083, loss=0.0002760275674518198
iter=116093, loss=0.0003045180637855083
iter=116103, loss=0.00031914407736621797
iter=116113, loss=0.0003321132971905172
iter=116123, loss=0.0005144361639395356
iter=116133, loss=0.0004932241281494498
iter=116143, loss=0.0003733484190888703
iter=116153, loss=0.0003277214418631047
iter=116163, loss=0.0003389561315998435
iter=116173, loss=0.00034767432953231037
iter=116183, loss=0.00038125179708004
iter=116193, loss=0.0003127865493297577
iter=116203, loss=0.00039649891550652683
iter=116213, loss=0.0003710694727487862
iter=116223, loss=0.0003599177289288491
iter=116233, loss=0.0003758402308449149
iter=116243, loss=0.0003532367991283536
iter=116253, loss=0.0003196400939486921
iter=116263, loss=0.00034279332612641156
iter=116273, loss=0.00034339589183218777
iter=116283, loss=0.000333048519678413

iter=118083, loss=0.00032922468381002545
iter=118093, loss=0.0003329329192638397
iter=118103, loss=0.0003726270515471697
iter=118113, loss=0.00036828158772550523
iter=118123, loss=0.0003048504004254937
iter=118133, loss=0.0002820089284796268
iter=118143, loss=0.00028916922747157514
iter=118153, loss=0.000433997658547014
iter=118163, loss=0.0003926354693248868
iter=118173, loss=0.0004766257479786873
iter=118183, loss=0.00040910622919909656
iter=118193, loss=0.00034560207859613
iter=118203, loss=0.0002955909294541925
iter=118213, loss=0.00029314120183698833
iter=118223, loss=0.0003192671574652195
iter=118233, loss=0.00035014940658584237
iter=118243, loss=0.0003624813980422914
iter=118253, loss=0.00039284827653318644
iter=118263, loss=0.00046372407814487815
iter=118273, loss=0.000379012490157038
iter=118283, loss=0.00037197174970060587
iter=118293, loss=0.000362986174877733
iter=118303, loss=0.0003392911166884005
iter=118313, loss=0.00038590398617088795
iter=118323, loss=0.000357553741196

iter=120113, loss=0.0002800204965751618
iter=120123, loss=0.0002696415758691728
iter=120133, loss=0.00026542440173216164
iter=120143, loss=0.00025511402054689825
iter=120153, loss=0.0002640279126353562
iter=120163, loss=0.0002941054117400199
iter=120173, loss=0.00031508441315963864
iter=120183, loss=0.0002899069804698229
iter=120193, loss=0.0003049336373806
iter=120203, loss=0.0002971566282212734
iter=120213, loss=0.0002758381888270378
iter=120223, loss=0.0002966348547488451
iter=120233, loss=0.0003355169319547713
iter=120243, loss=0.0002744614612311125
iter=120253, loss=0.00030140875605866313
iter=120263, loss=0.00036288792034611106
iter=120273, loss=0.00027595963911153376
iter=120283, loss=0.00028911710251122713
iter=120293, loss=0.0002404883853159845
iter=120303, loss=0.0003098247107118368
iter=120313, loss=0.0003439089050516486
iter=120323, loss=0.00027257075998932123
iter=120333, loss=0.0002629448426887393
iter=120343, loss=0.0002780199283733964
iter=120353, loss=0.000340362312272

iter=122143, loss=0.00037236622301861644
iter=122153, loss=0.00037143673398531973
iter=122163, loss=0.0003614001616369933
iter=122173, loss=0.0003146793460473418
iter=122183, loss=0.00028506567468866706
iter=122193, loss=0.00025723432190716267
iter=122203, loss=0.0002330746065126732
iter=122213, loss=0.00027299782959744334
iter=122223, loss=0.00030007059103809297
iter=122233, loss=0.0003098144952673465
iter=122243, loss=0.00031953418510966003
iter=122253, loss=0.00032958004157990217
iter=122263, loss=0.0003648041747510433
iter=122273, loss=0.0003386556345503777
iter=122283, loss=0.00030493130907416344
iter=122293, loss=0.00024662306532263756
iter=122303, loss=0.00025005254428833723
iter=122313, loss=0.0002768823178485036
iter=122323, loss=0.00028928881511092186
iter=122333, loss=0.00031684531131759286
iter=122343, loss=0.0003677485219668597
iter=122353, loss=0.00030967273050919175
iter=122363, loss=0.0002858303487300873
iter=122373, loss=0.00035354364081285894
iter=122383, loss=0.00029

...

iter=205123, loss=0.0002504816511645913
iter=205133, loss=0.0002701373887248337
iter=205143, loss=0.0002472562191542238
iter=205153, loss=0.00023523882555309683
iter=205163, loss=0.00022474484285339713
iter=205173, loss=0.0002581534208729863
iter=205183, loss=0.0002450634492561221
iter=205193, loss=0.00020853565365541726
iter=205203, loss=0.00022993070888333023
iter=205213, loss=0.000238289765547961
iter=205223, loss=0.00022480642655864358
iter=205233, loss=0.00021746203128714114
iter=205243, loss=0.0002180955489166081
iter=205253, loss=0.000242722817347385
iter=205263, loss=0.00022201352112460881
iter=205273, loss=0.00019776658155024052
iter=205283, loss=0.0002265617804368958
iter=205293, loss=0.00023836048785597086
iter=205303, loss=0.00023424401297234
iter=205313, loss=0.00026896921917796135
iter=205323, loss=0.00023632703232578933
iter=205333, loss=0.00021125726925674826
iter=205343, loss=0.00021374845528043807
iter=205353, loss=0.0002354996104259044
iter=205363, loss=0.00028324994

iter=207153, loss=0.00031890219543129206
iter=207163, loss=0.00027824152493849397
iter=207173, loss=0.00029817974427714944
iter=207183, loss=0.00027317763306200504
iter=207193, loss=0.0002828666474670172
iter=207203, loss=0.00024188507813960314
iter=207213, loss=0.0002615578123368323
iter=207223, loss=0.0003104070492554456
iter=207233, loss=0.00034858426079154015
iter=207243, loss=0.000312591262627393
iter=207253, loss=0.00030158355366438627
iter=207263, loss=0.0002620950108394027
iter=207273, loss=0.0002285178197780624
iter=207283, loss=0.00024240680795628577
iter=207293, loss=0.00026327124214731157
iter=207303, loss=0.00024329533334821463
iter=207313, loss=0.0002412661851849407
iter=207323, loss=0.00025166221894323826
iter=207333, loss=0.0002384670078754425
iter=207343, loss=0.00023163034347817302
iter=207353, loss=0.0002272968558827415
iter=207363, loss=0.0002530378114897758
iter=207373, loss=0.0002647335350047797
iter=207383, loss=0.00024667877005413175
iter=207393, loss=0.00021013

iter=209183, loss=0.0002978701377287507
iter=209193, loss=0.00025415242998860776
iter=209203, loss=0.00023405966931022704
iter=209213, loss=0.00022541903308592737
iter=209223, loss=0.0002649070811457932
iter=209233, loss=0.00028045184444636106
iter=209243, loss=0.0002778370399028063
iter=209253, loss=0.00032943589030764997
iter=209263, loss=0.00028678050148300827
iter=209273, loss=0.0002559373388066888
iter=209283, loss=0.0003044714976567775
iter=209293, loss=0.00026223252643831074
iter=209303, loss=0.0002785091055557132
iter=209313, loss=0.0002667632361408323
iter=209323, loss=0.00026316725416108966
iter=209333, loss=0.0002747614053077996
iter=209343, loss=0.0002671865513548255
iter=209353, loss=0.0002514912630431354
iter=209363, loss=0.00024529657093808055
iter=209373, loss=0.00023520662216469646
iter=209383, loss=0.00020601661526598036
iter=209393, loss=0.000248273485340178
iter=209403, loss=0.0002483420539647341
iter=209413, loss=0.0002277853200212121
iter=209423, loss=0.0002037463

iter=211213, loss=0.0002288851683260873
iter=211223, loss=0.00025537246256135404
iter=211233, loss=0.00023348809918388724
iter=211243, loss=0.00023301698092836887
iter=211253, loss=0.00025322611327283084
iter=211263, loss=0.0002636343124322593
iter=211273, loss=0.00024847668828442693
iter=211283, loss=0.00027875707019120455
iter=211293, loss=0.0003044348559342325
iter=211303, loss=0.0003597961040213704
iter=211313, loss=0.00027333496836945415
iter=211323, loss=0.00030382664408534765
iter=211333, loss=0.0002912628697231412
iter=211343, loss=0.0002852144534699619
iter=211353, loss=0.0002618987637106329
iter=211363, loss=0.00029157643439248204
iter=211373, loss=0.00023286044597625732
iter=211383, loss=0.00022538160555996
iter=211393, loss=0.0002293209545314312
iter=211403, loss=0.00026023408281616867
iter=211413, loss=0.00026899133808910847
iter=211423, loss=0.00024414362269453704
iter=211433, loss=0.00024604552891105413
iter=211443, loss=0.0003151193377561867
iter=211453, loss=0.00026397

iter=213243, loss=0.00022862905461806804
iter=213253, loss=0.00023021406377665699
iter=213263, loss=0.00019068311667069793
iter=213273, loss=0.00020844218670390546
iter=213283, loss=0.0001996161590795964
iter=213293, loss=0.00021197451860643923
iter=213303, loss=0.00024241788196377456
iter=213313, loss=0.0002670011017471552
iter=213323, loss=0.00025905101210810244
iter=213333, loss=0.00024679902708157897
iter=213343, loss=0.00026876485208049417
iter=213353, loss=0.00023695755226071924
iter=213363, loss=0.00024163695343304425
iter=213373, loss=0.00020770623814314604
iter=213383, loss=0.00021622050553560257
iter=213393, loss=0.0002504071453586221
iter=213403, loss=0.00028904067585244775
iter=213413, loss=0.00026540696853771806
iter=213423, loss=0.00030081943259574473
iter=213433, loss=0.0003491683164611459
iter=213443, loss=0.00033234411966986954
iter=213453, loss=0.0003001962322741747
iter=213463, loss=0.0003362347779329866
iter=213473, loss=0.00034176959889009595
iter=213483, loss=0.00